# PYDANTIC

## Qu'est-ce que pydantic ? A quoi sert-il ?

Pydantic est une bibliothèque qui utilise des annotations de type Python pour fournir des indications de type lors de l'exécution et pour fournir facilement des paramètres d'erreur lors de la validation des données.

Cette bibliothèque est utile avec SQLAlchemy pour vérifier et valider les données.

Cela nous évite de faire des fonctions pour vérifier que les variables soient du bon type et qu'il n'en manque pas.

## Comment l'installer ?

Il suffit d'un : pip install pydantic

## Comment l'utiliser ?

Nous avons repris ici une partie du code utilisé pour créer un utilisateur à l'aide de pydantic

Dans le fichier database.py :

In [ ]:
from sqlalchemy import create_engine
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker

SQLALCHEMY_DATABASE_URL = "sqlite:///./sql.db"

engine = create_engine(SQLALCHEMY_DATABASE_URL)

SessionLocal = sessionmaker(autocommit=False, autoflush=False, bind=engine)

Base = declarative_base()

Dans le fichier models.py :

In [1]:
from sqlalchemy import Integer, ForeignKey, String, Column, Boolean
from sqlalchemy.orm import relationship
from .database import Base 
import datetime

class User(Base):
    __tablename__ = 'users'
    id = Column(Integer, primary_key=True, index=True)
    is_admin = Column(Boolean, default=False)
    username = Column(String)
    password = Column(String, default="1234")

    notes = relationship("Note", back_populates="users")

class Note(Base):
    __tablename__ = 'notes'
    id = Column(Integer, primary_key=True, index=True)
    user_id = Column(Integer, ForeignKey('users.id'))
    date = Column(String, default=datetime.date.today().strftime("%d%m%Y"))
    note_content = Column(String(length=500))
    note_sentiment = Column(String)

    users = relationship("User", back_populates="notes")


Dans le fichier schemas.py :

In [ ]:
from typing import List, Optional
from pydantic import BaseModel

class NoteBase(BaseModel):
    date : str
    note_content: str
    note_sentiment: str 

class NoteUpdate(NoteBase):
    id : str
    user_id : str
    date: Optional[str] # Possibilité d'utiliser la fonctionnalité "Optional" 

class Note(NoteBase):
    id: int
    user_id: int

    class Config:
        orm_mode = True

class UserBase(BaseModel):
    username : str

class UserCreate(UserBase):
    is_admin : bool
    password: str 
    
# Quand on requête sur un user, on récupère ses attributs (Ici : id, is_admin et ses notes)
class User(UserBase): 
    id: int
    is_admin: bool
    notes: List[Note] = [] # Permet de récupérer les notes d'un user lorsqu'on requête un user

    class Config:
        orm_mode = True 
    # Permet de récupérer une donnée, même si ce n'est pas un dict, mais un ORM 
    # => user['id'] ou user.id reverront ainsi la donnée

Dans le fichier crud.py :

In [ ]:
from sqlalchemy.orm import Session
from . import models, schemas

# On attend un user de la forme UserCreate
def create_user(db: Session, user: schemas.UserCreate):
    db_user = models.User(is_admin=user.is_admin, username=user.username, password=user.password)
    db.add(db_user)
    db.commit()
    db.refresh(db_user)
    return db_user

Dans le fichier main.py :

In [ ]:
from fastapi import Depends, FastAPI
from sqlalchemy.orm import Session

from sqldb import crud, models, schemas
from sqldb.database import SessionLocal, engine

models.Base.metadata.create_all(bind=engine)

app = FastAPI()

def get_db():
    db = SessionLocal()
    try:
        yield db
    finally:
        db.close()

# Route pour créer un utilisateur - On attend que les informations envoyées 
# correspondent aux informations attendues dans UserCreate
@app.post("/users/", response_model=schemas.User) 
def create_user(user: schemas.UserCreate, db: Session = Depends(get_db)): 
    return crud.create_user(db=db, user=user)

## Ressources

https://fastapi.tiangolo.com/tutorial/sql-databases/#create-the-pydantic-models